In [8]:
import sqlite3

database = sqlite3.connect(r"C:\\Users\\swavi\\Documents\\GitHub\\F1-Stop-Strategy\\Databases\\database.db")
cursor = database.cursor()
driverIds = []
year = 2025

cursor.execute(
		"""SELECT raceID FROM Race WHERE year = ?""",
				(year,)
					)
races = [r[0] for r in cursor.fetchall()]


def getDrivers(raceId):
		cursor.execute(
			"""
			SELECT DISTINCT d.driverId, d.code
			FROM Driver d
			JOIN LAP l ON l.driverId = d.driverId
			WHERE l.raceId = ?
			""",
			(raceId,)
		)
		drivers = cursor.fetchall()
		return drivers


In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.ioff()
scaler = StandardScaler()


allLaps = []
allDrivers = []

columns = [
    "throttlePerc100",
    "throttlePerc0",
    "avCornerBrakeDistance",
    "throttleOscillation",
    "coastingPerc",
    "brakeChanges",
    "avSpeedCornerDiff",
    "avApexThrottle",
]


data = {col: [] for col in columns}

In [10]:
def collectingData(race, driverIds, driverCodes, averages):
  rows = []

  for driverId, driverCode in zip(driverIds, driverCodes):
    cursor.execute(
      """SELECT lapId
         FROM Lap 
         WHERE raceId = ? AND driverId = ? AND attacking = ? AND defending = ?""",
      (race, driverId, 0, 0)
    )

    lapIds = [laps[0] for laps in cursor.fetchall()]
    lapFeatures = []

    for lapId in lapIds:
      cursor.execute(
        f"""SELECT {",".join(columns)}
            FROM Features
            WHERE lapId = ?""",
        (lapId,)
      )

      row = cursor.fetchone()

      if row is None:
        continue
      if all(feature is not None for feature in row):
        lapFeatures.append(row)

    lapFeatures = np.array(lapFeatures, dtype=float)
    if lapFeatures.shape[0] == 0:
      continue

    if averages:
      values = lapFeatures.mean(axis=0)
      rows.append(
        dict(zip(columns, values), driver=driverCode, raceId=race)
      )
    else:
      rows.extend(
          dict(zip(columns, row), driver=driverCode, raceId=race)
          for row in lapFeatures)
  return pd.DataFrame(rows)


In [11]:
averages = False
rows = []

for race in races:
		drivers = getDrivers(race)
		driverIds = [row[0] for row in drivers]
		driverCodes = [row[1] for row in drivers]
		dfRace = (collectingData(race,driverIds,driverCodes,averages))
		rows.append(dfRace)

df = pd.concat(rows, ignore_index=True)
df[columns] = df.groupby("raceId")[columns].transform(
    lambda x: (x - x.mean()) / x.std()
)

In [12]:
from hdbscan import HDBSCAN

features = columns  
allLapsDF = df[features].copy()
scaler = StandardScaler()
scaled = scaler.fit_transform(allLapsDF)
pca = PCA(n_components=2)
Xpca = pca.fit_transform(scaled)
hdb = HDBSCAN(min_cluster_size=60, min_samples=30,
                      cluster_selection_method='leaf', metric='euclidean',prediction_data=True)
labels = hdb.fit_predict(Xpca)
from sklearn.metrics import silhouette_score
print("Silhouette score:", silhouette_score(Xpca, labels))
df = df.reset_index(drop=True) 
df["cluster"] = labels
clusterSummary = df.groupby("cluster")[features].mean()
clusterAssignment = pd.crosstab(df.driver, df.cluster)
clusterAssignmentPercentage = (clusterAssignment.div(clusterAssignment.sum(axis=0),axis=1) *100).round(2)
print(clusterAssignment)

Silhouette score: -0.45239806240046315
cluster   -1   0   1   2   3   4
driver                          
ALB      353   0   2   9   3  12
ALO      342   0   3   8   2  10
ANT      397   0   1  16   2  14
BEA      398   3   0   3   0   1
BOR      397   0   5   9   4  20
COL      184   0   2   2   1   5
DOO      151   0   0   4   2   9
GAS      316   0   5  11   5  11
HAD      379   0   7   4   8  18
HAM      444  21   0   0   0   1
HUL      384   0   9  11  13  18
LAW      390   0   5   7   3  19
LEC      451  26   0   0   0   0
NOR      449   0   5  17   1  24
OCO      427  22   0   0   0   0
PIA      440   1   9  16   3  26
RUS      428   1   1  13   3  10
SAI      391   0   2   6   0   7
STR      363   0   1   6   1   9
TSU      386   0   4   9   3  14
VER      439   0   3   6   7  24


In [13]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

features = columns
allLapsDF = df[features].copy()

# Scale
scaler = StandardScaler()
scaled = scaler.fit_transform(allLapsDF)

# Reduce to behaviour space
pca = PCA(n_components=2)
Xpca = pca.fit_transform(scaled)

# Cluster
kmeans = KMeans(n_clusters=7, random_state=42, n_init=20)
labels = kmeans.fit_predict(Xpca)

# Evaluate
print("Silhouette score:", silhouette_score(Xpca, labels))

# Attach to data
df = df.reset_index(drop=True)
df["cluster"] = labels

# Interpret
clusterSummary = df.groupby("cluster")[features].mean()
clusterAssignment = pd.crosstab(df.driver, df.cluster)
clusterAssignmentPercentage = (
        clusterAssignment.div(clusterAssignment.sum(axis=0), axis=1) * 100
).round(2)

print(clusterAssignment)

Silhouette score: 0.31965731978173967
cluster    0    1    2    3    4    5    6
driver                                    
ALB       15   50   26  169    1   54   64
ALO       29   53   26  132    0   58   67
ANT       27   62   24  170    0   63   84
BEA        0  192   77   76   12   25   23
BOR      129    6    4   89    0   57  150
COL       28   35   16   53    0   42   20
DOO        7   18    9   72    0   32   28
GAS       28   33   25  144    0   61   57
HAD      131    4    3  106    0  113   59
HAM        0   33   95    2  334    1    1
HUL      106   24    7  115    0   63  120
LAW      119    3    2  110    0  110   80
LEC        1   36  136   11  293    0    0
NOR       96   30    8  137    1   57  167
OCO        0  159  139    5  144    1    1
PIA       85   72    9  115    4   54  156
RUS       39   81   30  163    1   75   67
SAI       33  103   52   99    6   70   43
STR      168   21    9   63    1   44   74
TSU      108   15   12  105    0   87   89
VER      127   1

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = Xpca
y = df['driver']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
preds = rf.predict(X_test)

print("Driver identification accuracy:", accuracy_score(y_test, preds))

Driver identification accuracy: 0.12863849765258217


In [18]:
import joblib
import os

os.makedirs("models", exist_ok=True)

joblib.dump(scaler, "models/scaler.pkl")
joblib.dump(pca, "models/pca.pkl")
joblib.dump(kmeans, "models/kmeans.pkl")
joblib.dump(rf, "models/random_forest.pkl")
joblib.dump(clusterAssignmentPercentage, "models/cluster_map.pkl")
joblib.dump(columns, "models/columns.pkl")

results_bundle = {
    "df": df,
    "clusterAssignment": clusterAssignment,
    "clusterSummary": clusterSummary,
    "clusterAssignmentPercentage": clusterAssignmentPercentage,
    "Xpca": Xpca,
    "y_test": y_test,
    "preds": preds,
    "scaler": scaler,
    "pca": pca,
    "kmeans": kmeans,
    "rf": rf,
    "columns": columns,
    "labels": labels
}

joblib.dump(results_bundle, "models/results_bundle.pkl")

['models/results_bundle.pkl']